In [11]:
!pip install tweepy==3.10.0
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi

In [3]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [4]:
access_token = "1167574923910377472-HEEBHulTcXDgppNPdRz5eM8pxER28m"
access_secret = "WXcsSigZikNPljYzdstcJLOClUlp9Q6PT4OPudfzTQoCn"
consumer_key = "VMESOJPCpOk8HushVpIGX630t"
consumer_secret = "4ZJDZSclkiH403IProbpphuk7f0F11MAfyt6L2ejctGW0JWkOu"

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api =tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=False, compression=True)

def scraptweets(search_words, date_since, date_until):
    
    db_tweets = pd.DataFrame(columns=['username', 'tweetcreatedts', 'text'])
    tweets = tweepy.Cursor(
        api.search, q=search_words, lang="id",
        since=date_since, until=date_until, tweet_mode='extended').items(5000)
    tweet_list = [tweet for tweet in tweets]
    
    
    for tweet in tweet_list:
        username = tweet.user.screen_name
        tweetcreatedts = tweet.created_at
        
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
            
        ith_tweet = [username, tweetcreatedts, text]
        
        db_tweets.loc[len(db_tweets)] = ith_tweet
        print('Proses Scrapping Selesai Dengan Jumlah Data', len(db_tweets))
    filename = 'cashless.csv'
    db_tweets.to_csv(filename, index=False)
    
today = datetime.date.today().strftime("%Y-%m-%d")
last_week = datetime.date.today() - timedelta(360)
last_week = last_week.strftime("%Y-%m-%d")

search_words = "cashless"
date_since = "2019-1-1"
date_untill = today

scraptweets(search_words, date_since, date_untill)

TweepError: Twitter error response: status code = 401

In [6]:
#Read data
import pandas as pd
def load_data():
    data = pd.read_csv('cashless.csv')
    return data

In [7]:
tweet_df = load_data()

In [8]:
#buka tabel tweet yang berisi username, tweetcreatedts, dan text dari crawling data twitter
tweet_df = pd.DataFrame(tweet_df[['username', 'tweetcreatedts', 'text']])
tweet_df.head(10)

,username,tweetcreatedts,text
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless"
2,Ibn_aaminu,2023-01-31 21:29:12,Cashless policy ko cashless wahala.. Wai har m...
3,Dan_Mummy_,2023-01-31 21:11:18,Cashless policy ko cashless wahala.. Wai har m...
4,Abbakr_Abbaty,2023-01-31 20:54:44,Cashless policy ko cashless wahala.. Wai har m...
5,Ahmdsalihu,2023-01-31 20:53:10,Cashless policy ko cashless wahala.. Wai har m...
6,Ahmed_deedaht,2023-01-31 20:50:34,Cashless policy ko cashless wahala.. Wai har m...
7,GETshahmir,2023-01-31 20:22:09,@KamilWhisky Salam bro. Ada. Cashless admissio...
8,excellency004,2023-01-31 20:07:31,This Month fa Ansha Daurin Aure amma sadakin T...
9,SmokeWaggler,2023-01-31 19:30:46,Goodbye duit cash! Mulai Khamis ni kita dah ta...


In [9]:
# remove mention username dan Retweet
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    return text

In [10]:
tweet_df['clean_tweet']= np.vectorize(remove_pattern)(tweet_df['text'], " *RT* | *@[\w]*")
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless"
2,Ibn_aaminu,2023-01-31 21:29:12,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...
3,Dan_Mummy_,2023-01-31 21:11:18,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...
4,Abbakr_Abbaty,2023-01-31 20:54:44,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...


In [14]:
#remove simbol
def remove(text):
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return text
tweet_df['remove_http'] = tweet_df['clean_tweet'].apply(lambda x: remove(x))
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless
2,Ibn_aaminu,2023-01-31 21:29:12,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...
3,Dan_Mummy_,2023-01-31 21:11:18,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...
4,Abbakr_Abbaty,2023-01-31 20:54:44,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...


In [15]:
#remove
def remov(tweet):
    # remove stock market rickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    
    #remoce old style retweet text "RT"
    tweer = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hastags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    #remove angka
    tweet = re.sub('[0-9]+','', tweet)
    
    return tweet
tweet_df['remove_hastag'] = tweet_df['remove_http'].apply(lambda x: remove(x))
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless
2,Ibn_aaminu,2023-01-31 21:29:12,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...
3,Dan_Mummy_,2023-01-31 21:11:18,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...
4,Abbakr_Abbaty,2023-01-31 20:54:44,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...


In [66]:
tweet_df.drop_duplicates(subset = "remove_hastag", keep = 'first', inplace = True)
tweet_df.head(10)

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...,cashless polici cashless wahala wai har mai ka...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless,mbank rl udah cashless
2,GETshahmir,2023-01-31 20:22:09,@KamilWhisky Salam bro. Ada. Cashless admissio...,Salam bro. Ada. Cashless admission untuk hosp...,Salam bro Ada Cashless admission untuk hospita...,Salam bro Ada Cashless admission untuk hospita...,cashless admiss hospit lepa contest period tak...
3,excellency004,2023-01-31 20:07:31,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,thi month fa ansha daurin aur amma sadakin tra...
4,SmokeWaggler,2023-01-31 19:30:46,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash Mulai Khamis ni kita dah tak...,Goodbye duit cash Mulai Khamis ni kita dah tak...,goodby duit cash khami ni takkan terima duit c...
5,amirulhaziqqq,2023-01-31 18:40:23,@OwToTheZie Tapi itu terserah pada penjual jug...,"Tapi itu terserah pada penjual juga, cuma kad...",Tapi itu terserah pada penjual juga cuma kadar...,Tapi itu terserah pada penjual juga cuma kadar...,terserah penjual celik penggunaan cashless ni ...
6,notirfn,2023-01-31 18:09:51,aku ni dahlah cashless. tak suka pegang duit. ...,aku ni dahlah cashless. tak suka pegang duit. ...,aku ni dahlah cashless tak suka pegang duit bi...,aku ni dahlah cashless tak suka pegang duit bi...,ni dahlah cashless suka pegang duit time nak b...
7,kwetiawgoreng4,2023-01-31 18:03:49,@raanends_ @BANKBRI_ID enak ya bayar cashless,enak ya bayar cashless,enak ya bayar cashless,enak ya bayar cashless,bayar cashless
8,hoshwi,2023-01-31 17:54:38,"@ddiyeonm Eh buset, yang menerapkan cashless g...","Eh buset, yang menerapkan cashless gimana yah?",Eh buset yang menerapkan cashless gimana yah,Eh buset yang menerapkan cashless gimana yah,buset menerapkan cashless gimana
9,sadeeq_en,2023-01-31 17:24:25,@Basheer_bichi @BashirAhmaad Daga 10 Feb za'a ...,Daga 10 Feb za'a daina amfani da tsohon kudi ...,Daga 10 Feb za a daina amfani da tsohon kudi d...,Daga 10 Feb za a daina amfani da tsohon kudi d...,daga 10 feb za daina amfani da tsohon kudi don...


In [57]:
#import stopword

import nltk
nltk.download('stopwords', quiet=True)


from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopwords = ['yg', 'utk', 'cuman', 'deh', 'btw', 'tapi', 'tp', 'gua', 'gue', 'lo', 'lu', 'wts', 'wtb', 'wtt', 'apus',
                  'kalo', 'trs'  , 'jd', 'nih', 'ntar', 'lg', 'gk', 'ecusli', 'dpt', 'dr', 'kpn', 'kok', 'kyk',  'donk', 
                  'yah', 'u', 'ga', 'km', 'uh', 'huh', 'eh', 'bang', 'br', 'bro', 'kyk', 'rp ' , 'jt', 'khami', 'kan', 'gpp', 'sm', 'usah', 
                  'sob', 'sist', 'hiks', 'thx', 'ato', 'jg', 'gw', 'wkwk',  'mak', 'haha', 'ly', 'k', 'haha', 'dg', 'dri', 'duh', 'ye', 
                  'wkwkwk', 'syg', 'Btw', 'ges', 'gaes', 'guys', 'moga', 'kmrn', 'nemu', 'yukkk', 'wkwkw', 'klas', 'iw', 'ew', 'lho', 'sbnry', 
                  'org', 'gtu', 'bwt', 'klrga', 'clau', 'lbh', 'cpet', 'ku', 'wke', 'mba', 'mas', 'sdh', 'kmrn', 'spt', 'dlm', 'bs', 'krn', 'jgn', 
                  'sapa', 'spt', 'sh', 'wakakaka', 'sihhh', 'hehe', 'eh', 'dgn', 'la', 'kl', 'ttg', 'mn', 'kmna', 'kmn', 'tdk', 'tuh', 'dah', 'kek', 
                  'ko' , 'pls', 'bbrp', 'pd', 'mah', 'dhhh', 'kpd', 'tuh', 'kzl', 'byar', 'si', 'sii', 'cm', 'sy ', 'hahaha', 'weh', 'dlu', 'tuhh'
                 ]

data = stop_factory + more_stopwords

dictionary = ArrayDictionary(data)
str = StopWordRemover(dictionary)

print(data)

#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stremmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

# Happy Emoticons
emoticons_happy = set([':)', ':v', '<3', '>_<'])

# Sad Emoticons
emoticons_sad = set([':(', ':"', ':|'])

#all emoticons (happy+sad)
emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
    
    
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in data and # remove stopwords
               word not in emoticons and # remove emoticons
                   word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
            
    return tweets_clean

['a', 'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'arti', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagainamakah', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumla

In [36]:
from nltk.stem import PorterStemmer

In [37]:
stemmer = PorterStemmer()

In [60]:
tweet_df ['Tweet'] = tweet_df['remove_hastag'].apply(lambda x: clean_tweets(x))

In [61]:
tweet_df.head() #tokenization

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...,"[cashless, polici, cashless, wahala, wai, har,..."
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless,"[mbank, rl, udah, cashless]"
7,GETshahmir,2023-01-31 20:22:09,@KamilWhisky Salam bro. Ada. Cashless admissio...,Salam bro. Ada. Cashless admission untuk hosp...,Salam bro Ada Cashless admission untuk hospita...,Salam bro Ada Cashless admission untuk hospita...,"[cashless, admiss, hospit, lepa, contest, peri..."
8,excellency004,2023-01-31 20:07:31,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,"[thi, month, fa, ansha, daurin, aur, amma, sad..."
9,SmokeWaggler,2023-01-31 19:30:46,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash Mulai Khamis ni kita dah tak...,Goodbye duit cash Mulai Khamis ni kita dah tak...,"[goodby, duit, cash, khami, ni, takkan, terima..."


In [62]:
#remove punct
def remove_punct(text):
    text = " ".join([char for char in text if char not in string.punctuation])
    return text
tweet_df ['Tweet'] = tweet_df['Tweet'].apply(lambda x: remove_punct(x))

In [63]:
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...,cashless polici cashless wahala wai har mai ka...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless,mbank rl udah cashless
7,GETshahmir,2023-01-31 20:22:09,@KamilWhisky Salam bro. Ada. Cashless admissio...,Salam bro. Ada. Cashless admission untuk hosp...,Salam bro Ada Cashless admission untuk hospita...,Salam bro Ada Cashless admission untuk hospita...,cashless admiss hospit lepa contest period tak...
8,excellency004,2023-01-31 20:07:31,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,thi month fa ansha daurin aur amma sadakin tra...
9,SmokeWaggler,2023-01-31 19:30:46,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash Mulai Khamis ni kita dah tak...,Goodbye duit cash Mulai Khamis ni kita dah tak...,goodby duit cash khami ni takkan terima duit c...


In [64]:
#remove tweet kosong
tweet_df = tweet_df[tweet_df['Tweet']!='']
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...,cashless polici cashless wahala wai har mai ka...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless,mbank rl udah cashless
7,GETshahmir,2023-01-31 20:22:09,@KamilWhisky Salam bro. Ada. Cashless admissio...,Salam bro. Ada. Cashless admission untuk hosp...,Salam bro Ada Cashless admission untuk hospita...,Salam bro Ada Cashless admission untuk hospita...,cashless admiss hospit lepa contest period tak...
8,excellency004,2023-01-31 20:07:31,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,thi month fa ansha daurin aur amma sadakin tra...
9,SmokeWaggler,2023-01-31 19:30:46,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash Mulai Khamis ni kita dah tak...,Goodbye duit cash Mulai Khamis ni kita dah tak...,goodby duit cash khami ni takkan terima duit c...


In [65]:
#reset index
tweet_df = tweet_df.reset_index(drop=True)
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,Abdulilu1,2023-01-31 22:12:32,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala.. Wai har m...,Cashless policy ko cashless wahala Wai har mai...,Cashless policy ko cashless wahala Wai har mai...,cashless polici cashless wahala wai har mai ka...
1,xpppate,2023-01-31 21:59:47,"mau apus mbanking, rl udah biasa cashless","mau apus mbanking, rl udah biasa cashless",mau apus mbanking rl udah biasa cashless,mau apus mbanking rl udah biasa cashless,mbank rl udah cashless
2,GETshahmir,2023-01-31 20:22:09,@KamilWhisky Salam bro. Ada. Cashless admissio...,Salam bro. Ada. Cashless admission untuk hosp...,Salam bro Ada Cashless admission untuk hospita...,Salam bro Ada Cashless admission untuk hospita...,cashless admiss hospit lepa contest period tak...
3,excellency004,2023-01-31 20:07:31,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,This Month fa Ansha Daurin Aure amma sadakin T...,thi month fa ansha daurin aur amma sadakin tra...
4,SmokeWaggler,2023-01-31 19:30:46,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash! Mulai Khamis ni kita dah ta...,Goodbye duit cash Mulai Khamis ni kita dah tak...,Goodbye duit cash Mulai Khamis ni kita dah tak...,goodby duit cash khami ni takkan terima duit c...


In [12]:
# remove kolom
tweet_df.drop(tweet_df.columns[[0,1,2,3,4,5]], axis = 1, inplace = True)
tweet_df.head()

NameError: name 'tweet_df' is not defined

In [70]:
# simpan data bersih
tweet_df.to_csv('bersih_cashless.csv', encoding='utf8', index = False)

In [24]:
# install google
!pip3 install googletrans==3.1.0a0

In [17]:
import pandas as pd
import googletrans
from googletrans import Translator

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('bersih_cashless.csv')
df.head(10)

,Tweet
0,mau hapus mbanking biasa cashless
1,jarang cashless kalau naik gojek
2,pembayaran cashless belum merata di pombensin
3,penjual ingin tau cara pakai cashless
4,cashless tidak pegang uang
5,enak bayar cashless
6,menerapkan cashless
7,suka gaya hidup cashless rekomendasi card hold...
8,sekarang penjual cilok keliling kalau bisa bay...
9,diberitahukan kepada semua pembayar cukai untu...


In [20]:
translator = Translator()
translations = {}
for column in df.columns:
    # Unique elements of the column
    unique_elements = df[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element).text
translations

{'mau hapus mbanking biasa cashless': 'I want to remove ordinary cashless banking',
 'jarang cashless kalau naik gojek': "it's rarely cashless when you ride a motorcycle taxi",
 'pembayaran cashless belum merata di pombensin': 'Cashless payments are not evenly distributed at gas stations',
 'penjual ingin tau cara pakai cashless': 'the seller wants to know how to use cashless',
 'cashless tidak pegang uang': 'cashless does not hold money',
 'enak bayar cashless': 'nice to pay cashless',
 'menerapkan cashless': 'implement cashless',
 'suka gaya hidup cashless rekomendasi card holder premium': 'likes cashless lifestyle premium card holder recommendations',
 'sekarang penjual cilok keliling kalau bisa bayar pakai scan untuk cashless seneng banget': 'Now peekaboo sellers around if they can pay using a scan for cashless are really happy',
 'diberitahukan kepada semua pembayar cukai untuk melakukan pembayaran cukai tanah cashless': 'informed to all taxpayers to make cashless land tax payment

In [21]:
# mengganti semua kata yang diterjemahkan dari kamus ke original dataframe
df.replace(translations, inplace = True)
df.head(10)

,Tweet
0,I want to remove ordinary cashless banking
1,it's rarely cashless when you ride a motorcycl...
2,Cashless payments are not evenly distributed a...
3,the seller wants to know how to use cashless
4,cashless does not hold money
5,nice to pay cashless
6,implement cashless
7,likes cashless lifestyle premium card holder r...
8,Now peekaboo sellers around if they can pay us...
9,informed to all taxpayers to make cashless lan...


In [22]:
df.to_csv('translate_cashless.csv', encoding='utf8', index = False)